Background

This report summarizes some high-level findings about a bike-share arrangement in Washington DC (Capital Bike Share). Like bike share services in many major US cities, Capital Bike Share provides "docked" bikes placed at a large number of parking spots around the city.  Riders "check out" the bike from one station and then "check in" the bike at another (or the same) station.  Electronic equipment on the bike records the precise time that the bike leaves a dock and returns to a dock.  The bikes appear to have GPS capability as well, since each check-out and check-in now records (since the data from April 2020) the precise latitude and longitude of the dock (not just a general lat/long of the parking station).

In addition to learning more about the operation of a bike sharing service, I reinforced my understanding of some of techniques we've learned in the Thinkful course so far.  I've also learned a number of mechanical things, including

* How pull down CSV files off the web and pull them into a unified, normalized PostgreSQL database.
* How to clean up the data and prepare it for analysis.
* How accommodate large data sets by pushing the location of storage and retrieval of the data to other (larger, faster) computers.
* How to deploy Juypter Notebooks in a fashion where I have control and access to local data (unlike Google Colab) but also get around the size the speed restrictions of my laptop.

This report is organized into the following sections:

Data

Preliminary Tabulations and Observations

Statistical Tests

Wrap-up and ideas for further study

Appendices
    - Data cleanup
    - Technical issues related to manipulating and analyzing the data 

Parties potentially interested in this report include

* asdf
* ;lkj
* fdasjkl; 



Data

Primary Data Source

The list of suggested sites provided by Thinkful led me eventually to the Capital Bikeshare site, where they mention that they have been tracking each ride taken.  I went to that page and downloaded CSV files which were described as covering the period from 2010 through October of 2020.  I looked over these files and, where possible (see below for obstacles), bring them in to PostgreSQL database for easier viewing, error checking, and ability to manipulate further for analysis.

After manipulation and cleanup, I had the following two tables holding the ride data and accompanying station information:

bike_ride (about 25M records)



station_info(about xxx records)




Ancillary Data

I noted in my original proposal that Google Maps can provide an API which will give expected walking\biking\driving time given two locations (latitude and longitude) and time (time of day, day of week). I thought it would be interesting to compare actual observed biking times between popular bike stations with the time expected to be needed to walk or drive, and moreover to compare the actual bike times with those estimated by Google Maps.  To avoid extra cost, however, and keep a reasonable scope, I obtained travel times for only the top 15 routes by manual operation of the Google Maps site.



Preliminary Tabulations and Observations

Popularity of bike share over the years

    - Number of rides by year
    -Approximate size of fleet (number of individual bicycles)

Durability of the fleet (observed service life of bicycles)


Patterns of usage
    - Average ride length
    - volume of rides by time of day, weekday and weekend
    - distinction between regular users (with a subscription) versus casual (one-off rides).



Appendix - Data Manipulation and Cleanup

Pulling the data into PostgreSQL, I came across a number of obstacles which required some decisions:

*  The 2010 data showed no rides until 2010-09-16 (perhaps this was there first day of operation, although the volume of rides suggests not).  I decided to eliminate all data from 2010 (specifically, all rides beginning sometime in calendar 2010).
* Beginning in April of 2020, without calling attention to it, they decided to change the format of the data file COMPLETELY.  Furthermore, the file format continued to change slightly month-to-month for the rest of 2020.  I decided to eliminate 2020 entirely as a quick means of getting around this obstacle.  I noted, however, that the rides were significantly down in 2020 relative to 2019, as one would expect from Covid-19.
* The remaining data containing rides from 2011 to 2019 inclusive constituted about 25 million records and about 25G of data.  I decided to look for any ways that I could trim the size of the file without discarding information.  In addition to a start and end fields (timestamps with time of day as well as date), there was a "duration" field that indicated the number of seconds of ride from start to end.  Since this could be computed from the difference between the start and end timestamps, I performed a comparison of duration versus [end minus start].  In the vast majority of cases, they were equal (to within a second).  There were, however, a small number of situations where they were far apart (i.e., by more than 5 minutes) or nonsensical (end time was before start time).  Accordingly,  I decided to drop the end timestamp entirely and rely on the durations as provided.

In [ ]:
Proposal text:

Background

My son was an avid user of a bike share service (urban bike rental) when he was working in New York City.  I wanted to learn more about such services - perhaps how they’ve grown over time, how people use them (for commuting or for pleasure), and how they’ve been impacted by the Covid-19 pandemic.

I found a reference to a bike share database on the ‘Data is Plural’ site.  That data was old, however, limited to two calendar years, and it was grouped by day, which I felt would mask some interesting information about how bike usage varied during the day.  

However, that site provided a link over to the ‘Capital Bikeshare’ website (the company renting bikes in Washington DC) where I found the far more voluminous data I describe below.


Data Sources

Baseline data

Capital Bikeshare posts records of every single bike rental trip online here (zipped CSV files):

https://s3.amazonaws.com/capitalbikeshare-data/index.html

They started out (in 2010) providing annual files, but due to volume, they switched to monthly files beginning in 2018.  If the file formats are identical (or at least compatible), I plan to combine all data into one large file, which will cover the period 2010 through 2020 YTD (October right now) and permit me to see any interesting trends over time.

The fields in each file are as follows (this list is taken from the website):
Duration – Duration of trip
Start Date – Includes start date and time
End Date – Includes end date and time
Start Station – Includes starting station name and number
End Station – Includes ending station name and number
Bike Number – Includes ID number of bike used for the trip
Member Type – Indicates whether user was a "registered" member (Annual Member, 30-Day Member or Day Key Member) or a "casual" rider (Single Trip, 24-Hour Pass, 3-Day Pass or 5-Day Pass)

Examining the data a bit further, I note that the field Duration is in seconds.  (And since they provide duration, the ‘End Date’ field is actually redundant.) Unfortunately, there is no readily-available information on the demographics of the riders (as far as I can tell).

Data scrubbing and derived / ancillary data

I do not anticipate needing to perform any data cleanup (error checking, filling in for missing data, etc.), but I won’t know for sure until I assemble and review the data.

An obviously desirable piece of “derived” data would be the distance traveled between the start and end bike station.  I see that Google Maps actually provides a “Distance Travel Matrix API” that could give me this information:

https://developers.google.com/maps/documentation/distance-matrix/overview?csw=1

They charge for use of the API, however, so if the number of start/end station combinations isn’t too high, I should be able to get street distance (as well as latitude / longitude coordinates of each bicycle station) by scraping the Google Maps website instead.

The coordinates of the stations may permit me (if not too time-consuming) to construct maps of where the stations are located.  Also, if Google Maps can provide me with expected driving time between stations, it might be interesting to see how that compares with the average cycling time between stations (as exhibited by this data), to provide a proxy for how efficient cycling is as a means of getting around DC. (I do know from experience that car traffic there is really terrible.)

If Bike Number is unique to each bicycle (in particular, not re-used, at least not quickly), I may be able to assess the size of the bicycle fleet and how it has changed over time.


Hypotheses to test

Knowing what data I have to work with, I anticipate being able to test some hypotheses like the following.  (I won’t know for sure until I dig into the data a bit more, but I am reasonably confident that the data will yield some interesting insights.)

Bike usage follows a strong daily pattern on weekdays, with surges in the morning and evening.  Weekend patterns are completely different from weekday patterns.
Bike usage varies by time of year (a proxy for temperature and precipitation), with this pattern most strongly evident on the weekends, when cycling is primarily for pleasure rather than for commuting to/from work.
Use of bikes has grown strongly over time, but has taken a hit during the pandemic.
Growth in bike usage appears to lead (or perhaps trail?) the growth in the number of bikes available (and perhaps the number and size of the bike stations).
Registered riders are primarily commuters; casual riders are primarily tourists.
Certain stations are heavily used, and tend to run out of available bikes at peak hours.  Other stations are lightly used - they may have more bikes than they need, and they may be eliminated if usage gets too low.


Data Manipulation, Analysis Methods, Deliverable

To get answers to preliminary questions about the data, I anticipate bringing the CSV files together into a PostgreSQL database table, where I can perform SQL queries on it.  Ancillary information (like station locations and a distance matrix) will likely reside in other tables within the same database.

To perform any needed statistical tests, I anticipate bringing subsets (only the needed fields) of the data into Pandas dataframes and using the Python routines we’ve learned in the recent checkpoints.  I will likely be determining

Key statistics (rentals per hour, length of ride, etc.) and the means and standard deviations of those, and how those statistics have changed over time.
Correlations between statistics (e.g., time of day and the level of bike usage)
The results of inferential statistical tests against my hypotheses.  The appropriate tests will be chosen once I know more about my sample sizes and the apparent distributions (normal or not) of the data I’m seeing.

I hope to condense my report into a Jupyter Notebook, providing some sample code and graphics where appropriate.


Audience

Analysis of this bike rental data could be useful to many people, including

Managers of the bike rental company, to run the operation more efficiently.  For example, it might help them to identify where station capacity should be adjusted (increased or decreased), whether pricing should be adjusted to reflect bicycle demand (e.g., by weekday/weekend, or even by time of day), where and how the service should be advertised, etc.
City planners, to assess the growth in usage of bike rentals and how it could help address many urban problems, including automobile gridlock and pollution.
Commuters who currently use another form of transportation but who might be interested in trying bike rental.
